In [1]:
%matplotlib inline
import seaborn as sns
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

In [ ]:
df = pd.read_pickle("./data/processed/Cleaned_Chicago_Sales.pkl")
print("Data frame has {} rows and {} columns".format(len(df), len(df.columns)))
drop_these = []
for col in df.columns:
    if col[:4] == "Bool":
        drop_these.append(col)
df = df.drop(drop_these, axis=1)
print(list(df.columns))
df.head()

In [ ]:
i_vars = [
    'Age',
    'Age Decade',
    'Bedrooms',  
    'Building Square Feet', 
    'Estimate (Building)', 
    'Estimate (Land)', 
    'Fireplaces', 
    'Full Baths',
    'Garage 1 Area',
    'Garage 2 Area', 
    'Half Baths', 
    'Land Square Feet',
    'Lot Size Squared',
    'Rooms',
    'Sale Quarter'
    ]
dep_var = ['Sale Price']
X = df[i_vars]
y = df[dep_var]